In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

/home/lee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)
max_num_subprograms, max_cmd_len, max_act_len, max_actions_per_subprogram


(7, 10, 49, 9)

In [4]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap


In [5]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)
for li in commands:
    for wd in li:
        uni_commands.add(wd)
for li in actions:
    for wd in li:
        uni_actions.add(wd)
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')
num_cmd = len(uni_commands)
num_act = len(uni_actions)
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [6]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [7]:

commands_ind = [[command_map[c] for c in cmd] + [0] * (max_cmd_len - len(cmd)) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct) - 1)
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [8]:
tf.reset_default_graph()
default_sizes = 128
size_emb = 64
num_layers_encoder = 6
hidden_filters = 128
num_layers_subprogram = 3
hidden_filters_subprogram = default_sizes
cmd_mat = tf.Variable(1e-5*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(1e-5*tf.random_normal([num_act, size_emb]))
global_bs = None
global_time_len = 7
action_lengths = None
max_num_actions= None
# global_bs = 8
global_time_len = 7
max_num_actions = 9
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, 10,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs,))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))
learning_rate = tf.placeholder(tf.float32, shape = (None))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)
act_st_emb = tf.Variable(1e-5*tf.random_normal([size_emb]))
tf_bs = tf.shape(act_ind)[0]
act_st_emb_expanded = tf.tile(tf.reshape(
    act_st_emb, [1, 1, 1, size_emb]), [tf_bs, global_time_len, 1, 1])
act_emb_with_st = tf.concat((act_st_emb_expanded, act_emb), 2)

first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]

cells_subprogram_rnn = [first_cell_subprogram] + hidden_cells_subprogram

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    num_units=hidden_filters_encoder, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=hidden_filters_encoder//2, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
cells_subprogram = [
    tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanism, attention_layer_size = hidden_filters_subprogram) 
    for cell in cells_subprogram_rnn]

def subprogram(x, num_layers, cells, initial_states, lengths, reuse, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        print(idx)
        if idx == 0:
            num_past_units = hidden_filters
        else:
            num_past_units = hidden_filters_subprogram
        with tf.variable_scope(name + 'subprogram' + str(idx), reuse=reuse):
            self_attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units=num_past_units, memory=prev_layer,
                memory_sequence_length=lengths)
            cell_with_selfattention = tf.contrib.seq2seq.AttentionWrapper(
                    cells[idx], self_attention_mechanism, attention_layer_size = num_past_units)

            prev_layer, c = tf.nn.dynamic_rnn(
                    cell = cell_with_selfattention,
                    inputs = prev_layer,
                    sequence_length=lengths,
                    initial_state = None,
                    dtype=tf.float32,
                )
            prev_layer = tf.concat(prev_layer, 2)
            prev_layer = tf.nn.leaky_relu(prev_layer)
            returncells.append(c)
            hiddenlayers.append(prev_layer)
            if idx == num_layers - 1:
                output = tf.gather_nd(
                            prev_layer,
                            tf.stack([tf.range(bs), lengths], 1),
                            name=None
                        )
                return prev_layer, returncells, hiddenlayers, output
            prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms): 
    from_last_layer = tf.tile(tf.expand_dims(tf.concat((
        last_encoding, initial_cmb_encoding), 1), 1), [1, max_num_actions + 1, 1])
    autoregressive = act_emb_with_st[:,sub_idx, :, :]
    x_input = tf.concat((from_last_layer, autoregressive), -1)
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        x_input, num_layers_subprogram, cells_subprogram,None, 
        lengths = act_lengths[:, sub_idx], reuse = (sub_idx > 0), name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions + 1, num_act])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta = mlp(
        subprogram_output, [64], output_size = hidden_filters_encoder, name = 'global_transform',
        reuse = (sub_idx > 0)
    )
    last_encoding = last_encoding + delta
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
ppl_loss_avg = tf.reduce_mean(tf.pow(ppl_loss, 1.5)) * 100

tfvars = tf.trainable_variables()
weight_norm = tf.reduce_mean([tf.reduce_sum(tf.square(var)) for var in tfvars])*1e-3

action_taken = tf.argmax(act_presoftmax, -1, output_type=tf.int32)
correct_mat = tf.cast(tf.equal(action_taken, act_ind), tf.float32) * mask_ph
correct_percent = tf.reduce_sum(correct_mat, [1, 2])/tf.reduce_sum(mask_ph, [1, 2])
percent_correct = tf.reduce_mean(correct_percent)
percent_fully_correct = tf.reduce_mean(tf.cast(tf.equal(correct_percent, 1), tf.float32))

loss = ppl_loss_avg + weight_norm

opt_fcn = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
hidden_filters

128

In [10]:
encoding_last_layer

<tf.Tensor 'LeakyRelu_5/Maximum:0' shape=(?, 10, 256) dtype=float32>

In [11]:
np.random.seed(0)
trn_percent = .01
num_samples = mask_np.shape[0]
ordered_samples = np.arange(num_samples)
np.random.shuffle(ordered_samples)
trn_samples = ordered_samples[:int(np.ceil(num_samples*trn_percent))]
val_samples_all = ordered_samples[int(np.ceil(num_samples*trn_percent)):]
val_samples = val_samples_all
trn_samples.shape, val_samples.shape

((210,), (20700,))

In [12]:
eval_itr = -1
bs = 64
for itr in range(10000):
    samples = np.random.choice(trn_samples, size = bs, replace = False)
    if True:#itr == 0:
        trn_feed_dict = {
            cmd_ind : cmd_np[samples],
            act_ind : act_np[samples],
            mask_ph : mask_np[samples],
            act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[samples],
        }
        
    trn_feed_dict[learning_rate] = .01 / (np.sqrt(itr + 10))
    _, trn_loss, acc_trn_single, acc_trn = sess.run(
        [optimizer, loss, percent_correct, percent_fully_correct], trn_feed_dict)
    if itr == 0:
        trn_loss_avg = trn_loss
        acc_trn_avg = acc_trn
        acc_trn_single_avg = acc_trn_single
    else:
        trn_loss_avg = trn_loss_avg * .9 + trn_loss * .1
        acc_trn_avg = acc_trn_avg * .9 + acc_trn * .1
        acc_trn_single_avg = acc_trn_single_avg * .9 + acc_trn_single * .1
    if itr % 10 == 0 and itr > 0:
        # val_samples = np.random.choice(val_samples_all, size = bs, replace = False)
        eval_itr += 1
        val_feed_dict = {
            cmd_ind : cmd_np[val_samples],
            act_ind : act_np[val_samples],
            mask_ph : mask_np[val_samples],
            act_lengths : np.clip(struct_np[val_samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[val_samples]
        }
        val_loss, acc_val = sess.run([loss, percent_fully_correct], val_feed_dict)
        if eval_itr == 0:
            val_loss_avg = val_loss
            acc_val_avg = acc_val
        else:
            val_loss_avg = val_loss_avg * .9 + val_loss * .1
            acc_val_avg = acc_val_avg * .9 + acc_val * .1
        print('itr:', itr, 'trn_loss', trn_loss_avg, 'val_loss', val_loss_avg)
        print('itr:', itr, 'trn_acc', acc_trn_avg, 
              'trn_single_acc', acc_trn_single_avg, 'val_acc', acc_val_avg)

itr: 10 trn_loss 86.35061699876739 val_loss 74.25813
itr: 10 trn_acc 0.0 trn_single_acc 0.26231343246573324 val_acc 0.0
itr: 20 trn_loss 70.77462243636045 val_loss 72.83163757324219
itr: 20 trn_acc 0.0014062500000000002 trn_single_acc 0.43989527891308633 val_acc 0.00037681160029023886
itr: 30 trn_loss 57.89140280643862 val_loss 70.36643455505371
itr: 30 trn_acc 0.0026581735704531255 trn_single_acc 0.5631016794818173 val_acc 0.0013149758218787611
itr: 40 trn_loss 45.93048391539236 val_loss 68.32682555389405
itr: 40 trn_acc 0.02624175801718565 trn_single_acc 0.6395468136656974 val_acc 0.0030578744446393105
itr: 50 trn_loss 38.67291135754609 val_loss 64.61781224136354
itr: 50 trn_acc 0.05111564979466398 trn_single_acc 0.6986543609100662 val_acc 0.00915305289940443
itr: 60 trn_loss 27.31301435512408 val_loss 60.33090856544496
itr: 60 trn_acc 0.11431282712260134 trn_single_acc 0.7933211303985749 val_acc 0.018001032382098492
itr: 70 trn_loss 22.586914715508282 val_loss 55.60584173386384
itr:

itr: 520 trn_loss 0.052509035993110166 val_loss 1.26549057358151
itr: 520 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9628164591270107
itr: 530 trn_loss 0.05225979551474519 val_loss 1.1998601215509654
itr: 530 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9639116247186738
itr: 540 trn_loss 0.05200965872345383 val_loss 1.1639416447962596
itr: 540 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9648344742974351
itr: 550 trn_loss 0.05176377768403695 val_loss 1.1518060772137588
itr: 550 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9656747008312384
itr: 560 trn_loss 0.051518747890681005 val_loss 1.1208658047184419
itr: 560 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9664598904504156
itr: 570 trn_loss 0.05126978674085763 val_loss 1.088929095357514
itr: 570 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9671713890839019
itr: 580 trn_loss 0.051

itr: 1030 trn_loss 0.0427420968328177 val_loss 1.61677345280944
itr: 1030 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9757943305120313
itr: 1040 trn_loss 0.04259595588232411 val_loss 1.552633458183098
itr: 1040 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9758863943289618
itr: 1050 trn_loss 0.042450748059657775 val_loss 1.5611364607267575
itr: 1050 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9759064523104639
itr: 1060 trn_loss 0.04230173958094653 val_loss 1.5685439256655869
itr: 1060 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9759245044938158
itr: 1070 trn_loss 0.04215404662444983 val_loss 1.6101691258991686
itr: 1070 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9758924418942421
itr: 1080 trn_loss 0.04201072405233295 val_loss 1.6389583885411854
itr: 1080 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9758587516179343
itr: 1090 trn

itr: 1530 trn_loss 0.0363096801058542 val_loss 1.5022870986977241
itr: 1530 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9761477150451913
itr: 1540 trn_loss 0.036199784051344365 val_loss 1.468841149448069
itr: 1540 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9761899505196607
itr: 1550 trn_loss 0.036088390537378064 val_loss 1.4150123960125853
itr: 1550 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9762666100983557
itr: 1560 trn_loss 0.035975605800229606 val_loss 1.3712072163535722
itr: 1560 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9763211078695718
itr: 1570 trn_loss 0.03586251087490504 val_loss 1.3317495277267906
itr: 1570 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9763749898003575
itr: 1580 trn_loss 0.035751352325856904 val_loss 1.3208564601012063
itr: 1580 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9764138216251467
itr: 159

itr: 2030 trn_loss 0.031207020136490127 val_loss 1.9900022324775395
itr: 2030 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750697372642254
itr: 2040 trn_loss 0.031116373016544213 val_loss 2.0044199684430426
itr: 2040 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750748418230202
itr: 2050 trn_loss 0.031023927881606645 val_loss 2.0456840329619586
itr: 2050 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.975055278163408
itr: 2060 trn_loss 0.030933296001154006 val_loss 2.0610839962886875
itr: 2060 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750231809806121
itr: 2070 trn_loss 0.030839292019124664 val_loss 2.0281197345191635
itr: 2070 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750329411677683
itr: 2080 trn_loss 0.030743547657942856 val_loss 2.006821320885515
itr: 2080 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750127395974545
itr: 2

itr: 2530 trn_loss 0.02690056026354009 val_loss 1.935331304147628
itr: 2530 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9750974156933858
itr: 2540 trn_loss 0.02681934791150594 val_loss 1.8219561987591868
itr: 2540 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9752591709921808
itr: 2550 trn_loss 0.02673735122399446 val_loss 1.7276929688578622
itr: 2550 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9753950888481782
itr: 2560 trn_loss 0.026656988435468654 val_loss 1.6598965351694026
itr: 2560 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9754449535519224
itr: 2570 trn_loss 0.026575303616528184 val_loss 1.5978446386676906
itr: 2570 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.975494659761519
itr: 2580 trn_loss 0.026499452531479742 val_loss 1.6292559131165039
itr: 2580 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9754621000468111
itr: 2590

itr: 3030 trn_loss 0.023139932125835663 val_loss 0.8477648725505995
itr: 3030 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.976367607618384
itr: 3040 trn_loss 0.023069866612560188 val_loss 0.8351955560204084
itr: 3040 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9764216675505154
itr: 3050 trn_loss 0.022998144716923 val_loss 0.8281046879908468
itr: 3050 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9764606595765155
itr: 3060 trn_loss 0.022928260164128335 val_loss 0.8013876139480792
itr: 3060 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9765054143128337
itr: 3070 trn_loss 16.33170182179194 val_loss 2.22381747895708
itr: 3070 trn_acc 0.9253548593749998 trn_single_acc 0.9903347851461171 val_acc 0.9436954519144483
itr: 3080 trn_loss 11.738968238524993 val_loss 2.063551591046724
itr: 3080 trn_acc 0.85536190364958 trn_single_acc 0.9828188036873883 val_acc 0.9470602090053368
itr: 3090 trn_loss

KeyboardInterrupt: 

In [ ]:
correct_percent.shape, percent_fully_correct.shape

In [ ]:
10, None, 7, 9, None

In [ ]:
cmd_np.shape, act_np.shape, mask_np.shape, struct_np.shape, cmd_lengths_np.shape

In [ ]:
cmd_ind.shape, act_ind.shape, mask_ph.shape, act_lengths.shape, cmd_lengths.shape

In [ ]:
trn_samples.shape, val_samples.shape

In [ ]:

act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])

In [ ]:
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])

In [ ]:
max_actions_per_subprogram

In [ ]:
sess.run(act_presoftmax, feed_dict).shape

In [ ]:
action_map

In [ ]:
print(*actions_ind[2])

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape